## Скачиваем данные по imagewoof2

В ходе просмотра фотографий, а также изучения описания, стало ясно, что решаем задачу классификации породы собак.

In [ ]:
!wget https://s3.amazonaws.com/fast-ai-imageclas/imagewoof2.tgz

## Импортируем требуемые библиотеки

In [ ]:
import torch
import torchvision
from torchvision import transforms
from torchsummary import summary

import numpy as np
import matplotlib.pyplot as plt  # для отрисовки картиночек

import torch.nn as nn
import torch.nn.functional as F  # Functional


In [ ]:
import torch
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [ ]:
!ls imagewoof2

In [ ]:
!ls imagewoof2/train | wc

Разделяем данные на тренировочную и тестовую выборку, а также подгатавливаем наш pipeline для аугментации изображений. Такие возможны преобразования, которые написаны ниже:
<li>Изменение размера изображений (все изображения разного размера)</li>
<li>вырезание из изображения центальной части</li>
<li>афинное преобразование изображения - shift у изображения + поворот</li>
<li>зеркальный поворот по горизонтали</li>
<li>вырезать случайную часть изображения</li>
<li>закрасить определенную область изображения</li>
<li>нормализация изображения</li>

In [ ]:

train_transforms = transforms.Compose(
   [ transforms.Resize((250,250)), 
     transforms.CenterCrop(200), 
      #transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)), 
      transforms.RandomHorizontalFlip(), 
      #transforms.RandomCrop(32), 
      #transforms.RandomErasing(), 
      transforms.ToTensor(), #преобразование в тензор
      transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]) 
     ])

test_transforms = transforms.Compose([transforms.Resize((200, 200)),
                                      transforms.ToTensor(),
                                      #transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
                                     ])

trainset = torchvision.datasets.ImageFolder('imagewoof2/train', transform=train_transforms)
#trainset, valset = torch.utils.data.random_split(trainset, [7000, 2025])
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,shuffle=True, num_workers=1)

#valloader = torch.utils.data.DataLoader(valset, batch_size=16,
#                                          shuffle=True, num_workers=1)

testset = torchvision.datasets.ImageFolder('imagewoof2/val', transform=test_transforms)
testloader = torch.utils.data.DataLoader(testset, batch_size=16,
                                         shuffle=False, num_workers=1)
#какие породы собак мы распознаем:
classes = ['Australian terrier', 'Border terrier', 'Samoyed', 'Beagle', 'Shih-Tzu', 'English foxhound', 'Rhodesian ridgeback', 'Dingo', 'Golden retriever', 'Old English sheepdog']

## Пишем простую архитектуру нейросети:

In [ ]:
class SimpleConvNet(nn.Module):
    def __init__(self):
        # вызов конструктора предка nn.Module
        super(SimpleConvNet, self).__init__()
        # необходмо заранее знать, сколько каналов у картинки (сейчас = 3),
        # которую будем подавать в сеть, больше ничего
        # про входящие картинки знать не нужно
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5) #сверточный слой, ядро = 3х3, на выхое каналов - 6
        #self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2) #снижающий размерность поступивших на него данных
        self.bn1 = nn.BatchNorm2d(6) #слой, обеспечивающий пакетную (batch) нормализацию
        #self.relu = nn.ReLU(inplace=True) #активационная функция
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=3, kernel_size=3)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        #self.bn2 = nn.BatchNorm2d(12)
        #self.conv3 = nn.Conv2d(in_channels=12, out_channels=6, kernel_size=3)
        #self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(28227, 2048)  # !!!  #- fully-connected, Dense в Keras, linear - tensorflow
        self.drop = nn.Dropout(p=0.5)
        
        self.fc4 = nn.Linear(2048,  256)
        self.bn3 = nn.BatchNorm1d(256)
        #self.bn5 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 10)
        
        #self.max_batches_per_epoch=50

    def forward(self, x):
        #print(x.shape)
        x = F.relu(self.conv1(x))
        #x = self.conv1(x)
        x = self.bn1(x)
        #x = self.relu(x)
        x = self.pool2(F.relu(self.conv2(x)))
        #x=F.relu(self.conv3(x))
        #x = self.bn2(x)
        #x=self.pool3(F.relu(self.conv3(x)))
        #x = F.relu(self.conv3(x))
        #print(x.shape)
        
        x = x.view(-1, 28227)  # !!!
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = self.drop(x)
        
        x = self.fc4(x)
        
        #x = self.bn5(x)
        x = self.bn3(x)
        x = self.fc3(x)
        
        return x

In [ ]:
net = SimpleConvNet()

summary(net.cuda(), (3,200, 200))

In [ ]:
net = SimpleConvNet().cuda()

In [ ]:
# выбираем функцию потерь
loss_fn = torch.nn.CrossEntropyLoss()

# выбираем алгоритм оптимизации и learning_rate
learning_rate = 1e-3
optimizer = torch.optim.Adam(params = net.parameters(), lr=learning_rate) #оптимизатор
losses = []

In [ ]:
from tqdm import tqdm_notebook

## Обучаем нейронную сеть:

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(10,7))
ax = fig.add_subplot(1, 1, 1)
losses=[]
total_correct=0
total=0
# итерируемся
for epoch in tqdm_notebook(range(30)):
    mean_loss = 0
    batch_n = 0
    running_loss = 0.0
    print('Training')
    for i, batch in enumerate(tqdm_notebook(trainloader)):
        # так получаем текущий батч
        X_batch, y_batch = batch
        
        # обнуляем веса
        optimizer.zero_grad()
        #print(X_batch.size())
        # forward + backward + optimize
        y_pred = net(X_batch.cuda())
        #print(y_pred.size(),y_batch.size())
        loss = loss_fn(y_pred, y_batch.cuda())
        loss.backward()
        optimizer.step()
        mean_loss += float(loss)
        batch_n += 1
        # выведем текущий loss
        running_loss += loss.item()
        
        
        net.eval()
        max_arg_output = torch.argmax(y_pred, dim=1)
        total_correct += int(torch.sum(max_arg_output.cuda() == y_batch.cuda()))
        total += X_batch.shape[0]
        # выведем качество каждые 5 батчей
        if i % 5 == 4:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 5))
            losses.append(running_loss)
            running_loss = 0.0
    # Evaluationfor this fold
    print('Starting testing')
    mean_loss = 0
    batch_n = 0
    running_loss = 0.0
    total_correct = 0
    total = 0
    with torch.no_grad():
        for i, batch in enumerate(tqdm_notebook(valloader)):
            X_batch, y_batch = batch
            y_pred = net(X_batch.cuda())
            loss = loss_fn(y_pred, y_batch.cuda())
            mean_loss += float(loss)
            batch_n += 1
            running_loss += loss.item()
            max_arg_output = torch.argmax(y_pred, dim=1)
            total_correct += int(torch.sum(max_arg_output.cuda() == y_batch.cuda()))
            total += X_batch.shape[0]
            if i % 5 == 4:
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 5))
                losses.append(running_loss)
                running_loss = 0.0
    print('Test accuracy: {:.0%}'.format(total_correct/total))
    ax.clear()
    ax.plot(np.arange(len(losses)), losses)
    plt.show()

print('Обучение закончено')

In [ ]:
classes = ['Australian terrier', 'Border terrier', 'Samoyed', 'Beagle', 'Shih-Tzu', 'English foxhound', 'Rhodesian ridgeback', 'Dingo', 'Golden retriever', 'Old English sheepdog']

## Посчитаем качество распознавания для каждого класса:

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in tqdm_notebook(testloader):
        images, labels = data
        #print(images)
        y_pred = net(images.cuda())
        
        _, predicted = torch.max(y_pred, 1)
        
        c = (predicted.cpu().detach() == labels).squeeze()
        
        for i in range(len(labels)):
            
            label = labels[i]
            #print(labels)
            
            class_correct[label] += c[i].item()
            #print(class_correct)
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

## Выведем матрицу ошибок, чтобы посмотреть, на каких классах модель ошибается. В дальнейшем попробуем увеличить веса для "трудных" классов

In [ ]:
class_predict=[]
class_true=[]
with torch.no_grad():
    for data in tqdm_notebook(testloader):
        images, labels = data
        #print(images)
        y_pred = net(images.cuda())#.view(4, -1))
        
        _, predicted = torch.max(y_pred, 1)
        
        c = (predicted.cpu().detach() == labels).squeeze()
        
        for i in range(len(labels)):
            class_predict.append(predicted.cpu()[i])
            class_true.append(labels[i])

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(class_true,class_predict)

## ResNet

In [ ]:

train_transforms = transforms.Compose(
     [
      transforms.Resize((300,300)),
      transforms.CenterCrop(250),#224
      transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),
      #transforms.RandomHorizontalFlip(),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
     ])

test_transforms = transforms.Compose([transforms.Resize((250, 250)),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
                                     ])
trainset = torchvision.datasets.ImageFolder('imagewoof2/train', transform=train_transforms)
trainset, valset = torch.utils.data.random_split(trainset, [7000, 2025])
valloader = torch.utils.data.DataLoader(valset, batch_size=16,
                                          shuffle=True, num_workers=1)

valset.transform = test_transforms
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=1)

testset = torchvision.datasets.ImageFolder('imagewoof2/val', transform=test_transforms)
testloader = torch.utils.data.DataLoader(testset, batch_size=16,
                                         shuffle=False, num_workers=1)

classes = ['Australian terrier', 'Border terrier', 'Samoyed', 'Beagle', 'Shih-Tzu', 'English foxhound', 'Rhodesian ridgeback', 'Dingo', 'Golden retriever', 'Old English sheepdog']

In [ ]:
model1 = torchvision.models.resnet18(pretrained=False, num_classes=10)
# transfer the model to the GPU
model1 = model1.cuda()

# loss function
loss_fn = torch.nn.CrossEntropyLoss(weight = torch.Tensor([1.5,1.8,1.9,1.7,1.,1.,1.5,1.,1.,1.5]).cuda()) #указываем веса для кажого класса: для сложного класса, вес побольше

In [ ]:
learning_rate = 1e-3
# выбираем функцию потерь
# We'll optimize all parameters
optimizer = torch.optim.Adam(params = model1.parameters(), lr=learning_rate)

In [ ]:
%matplotlib inline
losses=[]
fig = plt.figure(figsize=(10,7))
ax = fig.add_subplot(1, 1, 1)
# итерируемся
for epoch in tqdm_notebook(range(10)):
    mean_loss = 0
    batch_n = 0
    running_loss = 0.0
    print('Training')
    for i, batch in enumerate(tqdm_notebook(trainloader)):
        # так получаем текущий батч
        X_batch, y_batch = batch
        
        # обнуляем веса
        optimizer.zero_grad()
        #print(X_batch.size())
        # forward + backward + optimize
        y_pred = model1(X_batch.cuda())
        
        #print(y_pred)
        loss = loss_fn(y_pred, y_batch.cuda())
        loss.backward()
        optimizer.step()
        mean_loss += float(loss)
        batch_n += 1
        # выведем текущий loss
        running_loss += loss.item()
        # выведем качество каждые 5 батчей        
        model1.eval()
    
        if i % 5 == 4:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 5))
            losses.append(running_loss)
            running_loss = 0.0
    print('Starting testing')
    total_correct = 0
    total = 0
    mean_loss = 0
    batch_n = 0
    running_loss = 0.0
    # Evaluationfor this fold
    correct, total = 0, 0
    with torch.no_grad():
        for i, batch in enumerate(tqdm_notebook(valloader)):
            X_batch, y_batch = batch
            y_pred = model1(X_batch.cuda())
            loss = loss_fn(y_pred, y_batch.cuda())
            mean_loss += float(loss)
            batch_n += 1
            # выведем текущий loss
            running_loss += loss.item()
            max_arg_output = torch.argmax(y_pred, dim=1)
            total_correct += int(torch.sum(max_arg_output.cuda() == y_batch.cuda()))
            total += X_batch.shape[0]
            if i % 5 == 4:
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 5))
                losses.append(running_loss)
                running_loss = 0.0
    print('Test accuracy: {:.0%}'.format(total_correct/total))
    ax.clear()
    ax.plot(np.arange(len(losses)), losses)
    plt.show()

print('Обучение закончено')